In [ ]:
# Install packages 
import os
import pandas as pd
import time
from transformers import pipeline
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
import undetected_chromedriver as uc
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException

In [ ]:
# URL of Forex Factory News
chrome_options = Options()
url = "https://www.forexfactory.com/news"
chrome_options.add_argument("--disable-blink-features=AutomationControlled") 

In [ ]:
ua = UserAgent()
chrome_options.add_argument(f"user-agent={ua.random}")

In [ ]:
driver = uc.Chrome()
driver.get(url)
time.sleep(3)

In [ ]:
driver.get("https://www.ig.com/en/tag/forex")

In [ ]:
try:
    # Wait for the cookie popup and click "Accepter"
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accepter')]"))
    )
    accept_button.click()
    print("Cookie banner accepted.")
    time.sleep(2)  # Wait for any animations to complete

except Exception as e:
    print("No cookie popup found or error occurred:", e)

In [ ]:


# Number of times to attempt clicking the button
max_attempts = 150

for attempt in range(max_attempts):
    try:
        print(f"Attempt {attempt + 1}/{max_attempts}")

        # Wait for the button to be clickable
        more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'more-results-button')]"))
        )

        # Add a random delay
        time.sleep(3)

        # Scroll to the button
        driver.execute_script("arguments[0].scrollIntoView();", more_button)
        
        # Add another random delay
        time.sleep(3)

        # Click the button
        driver.execute_script("arguments[0].click();", more_button)

        # Random delay after clicking
        time.sleep(3)

    except (TimeoutException, NoSuchElementException, ElementClickInterceptedException) as e:
        print(f"Exception occurred on attempt {attempt + 1}: {e}")
        print("Button is no longer available or page has changed. Stopping script.")
        break  # Exit the loop when the button is not found or not clickable

In [ ]:
response = driver.page_source
soup = BeautifulSoup(response, "html.parser")
articles=[]
for var in soup.find_all('h2',class_="article-result__title"):
    articles.append(var.get_text(strip=True))
articles.remove('Article result template')

In [ ]:
# Load FinBERT model
finbert = pipeline("sentiment-analysis", model="ProsusAI/finbert")
# Function to classify sentiment using FinBERT
def classify_sentiment(text):
    result = finbert(text)[0] 
    return result["label"]  

In [ ]:
df = pd.read_csv("data/forex_news.csv")
df = pd.concat([df, pd.DataFrame({'news': articles, 'labels': list(map(classify_sentiment, articles))})], axis=0, ignore_index=True)
df = df.drop_duplicates().reset_index(drop=True)  

In [ ]:
df['news'] = df['news'].fillna('')
df=df[['news', 'labels']]
df.to_csv("data/forex_news.csv")